<a href="https://colab.research.google.com/github/AntonioHenriqueAC/tcc/blob/master/google_colab/Barcode_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baixando as bibliotecas necessárias

In [0]:
!pip3 install tensorflow-gpu==1.13.1

In [0]:
!pip3 install imageai --upgrade

# Conectando o drive ao colab #

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Treinar novo Modelo !

## 1. Baixando a CNN pré-treinada ##


In [0]:
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

## 2. Carregando os arquivos do drive no colab ##

In [0]:
!unzip "/content/drive/My Drive/Colab/datasets/barcode4.zip" 

## 3. Treinando o modelo ##

In [0]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="barcode")
trainer.setTrainConfig(object_names_array=["barcode"], batch_size=4, num_experiments=11, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

## 4. Avaliando os modelos gerados ##

In [0]:
metrics = trainer.evaluateModel(model_path="barcode/models", json_path="barcode/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

# Utilizar modelo já treinado.

## 1. Criando as pastas ##

In [0]:
import os

path, dirs, files = next(os.walk("/content/drive/My Drive/Colab/Corridas"))
file_count_corridas = len(dirs)
file_count_corridas = file_count_corridas - 1
print("Número de corridas na pasta: ", file_count_corridas)


newFolderCorrida = str(file_count_corridas)
folderCorrida = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida
!mkdir $folderCorrida

folderTagsJson = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/tags_json"
!mkdir $folderTagsJson



## 2. Setando as variaveis ##


In [0]:
modelPath = "/content/drive/My Drive/Colab/models/barcode/detection_model-ex-007--loss-0003.066.h5"
JsonPath = "drive/My Drive/Colab/json/detection_config_barcode.json"
InputImage = "drive/My Drive/Colab/Imagens_Originais/img1.jpg"

## 3. Treinando modelo e recortando os codigos de barras da imagem original ##


In [0]:
from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(modelPath)
detector.setJsonPath(JsonPath)
detector.loadModel()

detections, extracted_objects_array = detector.detectObjectsFromImage(input_image = InputImage, 
                                                                      output_image_path="barcode-detected.jpg",
                                                                      minimum_percentage_probability=30,
                                                                      extract_detected_objects=True)

print("Número       " , ": ", "Porcentagem ", "      :    ", "box_points")
print("----------------------------------------------------------------")

print("")

for detection, object_path in zip(detections, extracted_objects_array):
    print(detection["name"], "      : ", detection["percentage_probability"], " : ", detection["box_points"])


print("----------------------------------------------------------------")
print("Quantidade de código de barras detectados na imagem: ", len(detections))

## 4. Movendo as pastas para o diretório correto. ##


In [0]:
!mv /content/barcode-detected.jpg $folder
!mv /content/barcode-detected-objects $folder

barcode = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/barcode-detected-objects"
images = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/tags_images"
!mv  $barcode $images